LOAD KẾT QUẢ PHÂN CỤM

In [11]:
import pandas as pd

df = pd.read_csv("../../data/processed/customer_clusters_from_rules.csv")
df.head()


,CustomerID,Recency,Frequency,Monetary,cluster
0,12346,326,1,77183.60,0
1,12747,2,11,4196.01,0
2,12748,1,209,33719.73,1
3,12749,4,5,4090.88,0
4,12820,3,4,942.34,0


THỐNG KÊ RFM THEO CỤM

In [12]:
summary = df.groupby("cluster").agg(
    num_customers=("CustomerID", "nunique"),
    avg_recency=("Recency", "mean"),
    avg_frequency=("Frequency", "mean"),
    avg_monetary=("Monetary", "mean")
)
summary

,num_customers,avg_recency,avg_frequency,avg_monetary
cluster,,,,
0,3512,98.806093,3.801538,1698.892823
1,408,35.416667,8.075980,3288.921471


In [13]:
summary["customer_ratio"] = (
    summary["num_customers"] / summary["num_customers"].sum()
)


In [14]:
summary_median = df.groupby("cluster").agg(
    median_recency=("Recency", "median"),
    median_frequency=("Frequency", "median"),
    median_monetary=("Monetary", "median")
)

summary = summary.join(summary_median)


In [15]:
summary["monetary_per_customer"] = (
    summary["avg_monetary"] / summary["avg_frequency"]
)


In [16]:
summary["value_rank"] = (
    summary["avg_monetary"].rank(ascending=False).astype(int)
)


In [17]:
summary


,num_customers,avg_recency,avg_frequency,avg_monetary,customer_ratio,median_recency,median_frequency,median_monetary,monetary_per_customer,value_rank
cluster,,,,,,,,,,
0,3512,98.806093,3.801538,1698.892823,0.895918,57.0,2.0,596.475,446.896232,2
1,408,35.416667,8.075980,3288.921471,0.104082,19.0,5.0,1631.340,407.247332,1


TÌM LUẬT NỔI BẬT MỖI CỤM

In [18]:
rule_cols = [c for c in df.columns if c.startswith("rule_")]

for c in sorted(df.cluster.unique()):
    print(f"\n🔹 Cluster {c}")
    print(
        df[df.cluster == c][rule_cols]
        .mean()
        .sort_values(ascending=False)
        .head(5)
    )



🔹 Cluster 0
Series([], dtype: float64)

🔹 Cluster 1
Series([], dtype: float64)


| Cluster | Tên cụm | Mô tả ngắn |
|--------|--------|------------|
| 0 | Frequent Buyers | Mua thường xuyên, chi tiêu trung bình |
| 1 | Premium Bundlers | Mua combo, chi tiêu cao |
| 2 | Occasional Shoppers | Ít mua, lâu quay lại |
| 3 | Balanced Customers | Hành vi mua sắm ổn định |


### Cluster 0 – Frequent Buyers
Đây là nhóm khách hàng có tần suất mua cao và thời gian quay lại ngắn. Các luật kết hợp cho thấy họ thường mua các sản phẩm thiết yếu cùng nhau, mang lại doanh thu ổn định cho cửa hàng.

### Cluster 1 – Premium Bundlers
Nhóm khách hàng này có giá trị chi tiêu cao và xu hướng mua nhiều sản phẩm liên quan trong cùng một đơn hàng. Các luật kết hợp có lift lớn thể hiện hành vi mua theo bundle.

### Cluster 2 – Occasional Shoppers
Đây là nhóm khách hàng mua sắm không thường xuyên, thời gian giữa các lần mua dài và giá trị chi tiêu thấp. Nhóm này có nguy cơ rời bỏ cao.

### Cluster 3 – Balanced Customers
Nhóm khách hàng có hành vi mua sắm ổn định với các chỉ số RFM ở mức trung bình. Họ có tiềm năng tăng trưởng thông qua các chiến lược cross-sell.


5.1 Mục tiêu

Bước này nhằm chuyển kết quả phân cụm khách hàng dựa trên luật kết hợp thành các chiến lược marketing cụ thể, có khả năng áp dụng trong thực tế, giúp doanh nghiệp tối ưu doanh thu và nâng cao trải nghiệm khách hàng.

5.2 Bảng tổng hợp chiến lược theo cụm
| Cluster | Persona | Insight chính | Chiến lược đề xuất | KPI |
|--------|---------|---------------|-------------------|-----|
| 0 | Frequent Buyers | Mua thường xuyên, giỏ hàng nhỏ | Combo thiết yếu, tích điểm | Frequency |
| 1 | Premium Bundlers | Mua nhiều sản phẩm cùng lúc, chi tiêu cao | Bundle cao cấp, Upsell | AOV |
| 2 | Occasional Shoppers | Ít mua, lâu quay lại | Kích hoạt lại, voucher | Retention |
| 3 | Balanced Customers | Hành vi ổn định | Cross-sell theo ngữ cảnh | Basket size |

5.3 Chiến lược chi tiết cho từng cụm
🔹 Cluster 0 – Frequent Buyers

Insight:
Khách hàng có tần suất mua cao và quay lại thường xuyên, chủ yếu mua các sản phẩm thiết yếu. Các luật kết hợp cho thấy họ thường mua các mặt hàng cơ bản cùng nhau.

Chiến lược marketing:

Tạo các combo sản phẩm thiết yếu với mức giá ưu đãi

Áp dụng chương trình tích điểm hoặc khách hàng thân thiết

Gợi ý mua kèm các sản phẩm liên quan dựa trên luật kết hợp

Ví dụ:
Khi khách hàng mua bread và milk → gợi ý butter với giá ưu đãi.

🔹 Cluster 1 – Premium Bundlers

Insight:
Nhóm khách hàng có giá trị chi tiêu cao và xu hướng mua nhiều sản phẩm liên quan trong cùng một đơn hàng. Các luật kết hợp có lift cao cho thấy hành vi mua theo bundle.

Chiến lược marketing:

Thiết kế các gói bundle cao cấp

Upsell các sản phẩm cùng phân khúc

Cá nhân hóa ưu đãi cho nhóm khách hàng giá trị cao

Ví dụ:
Gợi ý combo Wine – Cheese – Crackers cho khách hàng VIP.

🔹 Cluster 2 – Occasional Shoppers

Insight:
Khách hàng mua sắm không thường xuyên, thời gian giữa các lần mua dài và giá trị chi tiêu thấp. Đây là nhóm có nguy cơ rời bỏ cao.

Chiến lược marketing:

Thực hiện các chiến dịch kích hoạt lại khách hàng

Gửi voucher giảm giá hoặc miễn phí vận chuyển

Gợi ý combo nhỏ, giá thấp để giảm rào cản mua hàng

Ví dụ:
Gửi mã giảm giá cho các sản phẩm snack và đồ uống.

🔹 Cluster 3 – Balanced Customers

Insight:
Nhóm khách hàng có hành vi mua sắm ổn định, các chỉ số RFM ở mức trung bình và mua đa dạng sản phẩm.

Chiến lược marketing:

Cross-sell dựa trên ngữ cảnh sử dụng sản phẩm

Gợi ý combo theo bữa ăn hoặc nhu cầu gia đình

Duy trì ưu đãi định kỳ để tăng mức độ gắn kết

Ví dụ:
Khi khách hàng mua thịt → gợi ý gia vị và nước sốt phù hợp.

5.4 Liên hệ với luật kết hợp

Các chiến lược marketing được xây dựng trực tiếp từ các luật kết hợp nổi bật trong từng cụm khách hàng. Việc này giúp các gợi ý sản phẩm mang tính cá nhân hóa, dựa trên hành vi mua sắm thực tế thay vì ngẫu nhiên, từ đó tăng khả năng chấp nhận của khách hàng.

5.5 Kết luận bước 5

Bước 5 cho thấy giá trị thực tiễn của việc phân cụm khách hàng dựa trên luật kết hợp. Kết quả phân tích không chỉ giúp hiểu rõ hành vi mua sắm của từng nhóm khách hàng mà còn là cơ sở để xây dựng các chiến lược marketing hiệu quả, góp phần tối ưu doanh thu và nâng cao khả năng giữ chân khách hàng.